In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/Almabetter/Projects/Stock Price Prediction/GOOG.csv'

In [ ]:
df = pd.read_csv(file_path)
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-07-02,2536.790039,2576.959961,2535.379883,2574.379883,2574.379883,1058000
1,2021-07-06,2588.989990,2597.689941,2568.179932,2595.419922,2595.419922,1067500
2,2021-07-07,2606.820068,2612.798096,2595.199951,2601.550049,2601.550049,834000
3,2021-07-08,2565.000000,2600.649902,2560.800049,2583.540039,2583.540039,989000
4,2021-07-09,2578.889893,2596.989990,2578.870117,2591.489990,2591.489990,755300
...,...,...,...,...,...,...,...
247,2022-06-27,2378.699951,2385.000000,2320.014893,2332.449951,2332.449951,1642000
248,2022-06-28,2327.020020,2357.129883,2248.879883,2251.429932,2251.429932,1411600
249,2022-06-29,2242.969971,2273.290039,2231.080078,2245.129883,2245.129883,931400
250,2022-06-30,2209.989990,2226.595947,2146.199951,2187.449951,2187.449951,1902300


In [ ]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [ ]:
df.shape

(252, 7)

In [ ]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-07-02,2536.790039,2576.959961,2535.379883,2574.379883,2574.379883,1058000
1,2021-07-06,2588.989990,2597.689941,2568.179932,2595.419922,2595.419922,1067500
2,2021-07-07,2606.820068,2612.798096,2595.199951,2601.550049,2601.550049,834000
3,2021-07-08,2565.000000,2600.649902,2560.800049,2583.540039,2583.540039,989000
4,2021-07-09,2578.889893,2596.989990,2578.870117,2591.489990,2591.489990,755300


In [ ]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,252.000000,252.000000,252.000000,252.000000,252.000000,2.520000e+02
mean,2685.363434,2714.195279,2654.224208,2683.406235,2683.406235,1.321835e+06
std,234.848094,228.665885,236.058349,232.676748,232.676748,5.386850e+05
min,2102.840088,2127.899902,2044.160034,2116.790039,2116.790039,5.933000e+05
25%,2598.942444,2619.908753,2551.157532,2585.010071,2585.010071,9.461000e+05
50%,2742.252076,2768.175049,2714.890014,2740.405029,2740.405029,1.198000e+06
75%,2864.472534,2886.227478,2834.732483,2856.075074,2856.075074,1.552425e+06
max,3037.270020,3042.000000,2997.750000,3014.179932,3014.179932,4.487500e+06


In [ ]:
df.corr()

,Open,High,Low,Close,Adj Close,Volume
Open,1.000000,0.994161,0.993127,0.984332,0.984332,-0.315149
High,0.994161,1.000000,0.992898,0.992542,0.992542,-0.294050
Low,0.993127,0.992898,1.000000,0.993700,0.993700,-0.370369
Close,0.984332,0.992542,0.993700,1.000000,1.000000,-0.344281
Adj Close,0.984332,0.992542,0.993700,1.000000,1.000000,-0.344281
Volume,-0.315149,-0.294050,-0.370369,-0.344281,-0.344281,1.000000


In [ ]:
figure = go.Figure(data=[go.Candlestick(x=df["Date"], open= df["Open"], high=df["High"], low=df["Low"], close=df["Close"])])

In [ ]:
figure.update_layout(title = "Google Stock Price Analysis", xaxis_rangeslider_visible=False)

figure.show()

# Data Preparation

In [ ]:
def prepare_data(df, forecast_col, forecast_out, test_size): 
  label = df[forecast_col].shift(-forecast_out)  #creating new column called label with the last 5 rows are nan
  x = np.array(df[[forecast_col]])  #creating the feature array
  x = preprocessing.scale(x)  #processing the feature array
  x_lately = x[-forecast_out:] #creating the column i want to use later in the predicting method  
  x = x[:-forecast_out]  # X that will contain the training and testing
  label.dropna(inplace=True) #dropping na values
  y = np.array(label)  # assigning Y
  
  x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=0) #cross validation

  response = [x_train,x_test, y_train, y_test, x_lately]

  return response

  

In [ ]:
forecast_col = 'Close'
forecast_out = 5
test_size = 0.2

In [ ]:
x_train, x_test, y_train, y_test, x_lately = prepare_data(df, forecast_col, forecast_out, test_size);  #calling the method were the cross validation and data preperation is in
learner = LinearRegression()  #initializing linear regression model

learner.fit(x_train, y_train)  #training the linear regression model

LinearRegression()

In [ ]:
learner.intercept_

2684.6155830536977

In [ ]:
learner.coef_

array([216.54306773])

In [ ]:
y_pred_train = learner.predict(x_train)

In [ ]:
score = learner.score(x_test,y_test) #testing the linear regression model
forecast = learner.predict(x_lately) #set that will contain the forecasted data
response = {}  #creting json object
response['test_score'] = score
response['forecast_set'] = forecast

response

{'forecast_set': array([2357.34442537, 2281.79223672, 2275.91736164, 2222.13009823,
        2216.69370456]), 'test_score': 0.7766966936035584}

In [43]:
y_pred

array([2967.74859645, 2943.22346935, 2848.55477239, 2955.98017833,
       2754.17520841, 2749.5779847 , 2798.51622828, 2672.23521706,
       2683.86384968, 2849.58996115, 2368.6652561 , 2512.4213166 ,
       2914.65138397, 2896.27136944, 2574.11626822, 2786.80381499,
       2821.71703269, 2848.35898111, 2864.62194585, 2727.36547113,
       2202.05307488, 2359.6570389 , 2746.48197945, 2913.5975263 ,
       2882.39569104, 2813.16596487, 2840.3486158 , 2883.5800007 ,
       2630.45883437, 2712.24947824, 2993.06622312, 2905.28892109,
       2480.94901632, 2617.35698664, 2719.27132516, 2943.73639698,
       2655.33820716, 2286.03521381, 2713.11665154, 2863.77344144,
       2724.1576828 , 2767.27705615, 2786.74781015, 2367.31293099,
       2615.55730166, 2962.34931213, 2292.78659665, 2918.62389698,
       2677.056462  , 2728.82024581])

In [44]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from numpy import math

In [45]:
mean_squared_error(y_test, y_pred)

11212.456625640016

In [46]:
math.sqrt(mean_squared_error(y_train, y_pred_train))

102.97725306279504

In [47]:
r2_score(y_train, y_pred_train)

0.8105845626067134

In [48]:
r2_score(y_test, y_pred)

0.7766966936035584

# Now let's have a look to the another method which is used to predict the stock prices for next 5 days with the help of Autots Library


In [ ]:
pip install autots

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 486 kB 5.1 MB/s 


In [ ]:
from autots import AutoTS


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
model = AutoTS(forecast_length=5, frequency='infer', ensemble='simple')

In [ ]:
model = model.fit(df, date_col = 'Date', value_col='Close',id_col = None)


Inferred frequency is: None
Frequency is 'None'! Input frequency not recognized.
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model DatepartRegression in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Model Number: 6 with model DatepartRegression in generation 0 of 10


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Model Number: 7 with model DatepartRegression in generation 0 of 10
Epoch 1/50
12/12 [==============================] - 13s 9ms/step - loss: 0.4042
Epoch 2/50
12/12 [==============================] - 0s 8ms/step - loss: 0.3897
Epoch 3/50
12/12 [==============================] - 0s 8ms/step - loss: 0.3762
Epoch 4/50
12/12 [==============================] - 0s 9ms/step - loss: 0.3582
Epoch 5/50
12/12 [==============================] - 0s 9ms/step - loss: 0.3463
Epoch 6/50
12/12 [==============================] - 0s 10ms/step - loss: 0.3331
Epoch 7/50
12/12 [==============================] - 0s 9ms/step - loss: 0.3224
Epoch 8/50
12/12 [==============================] - 0s 9ms/step - loss: 0.3184
Epoch 9/50
12/12 [==============================] - 0s 9ms/step - loss: 0.3118
Epoch 10/50
12/12 [==============================] - 0s 9ms/step - loss: 0.3040
Epoch 11/50
12/12 [==============================] - 0s 10ms/step - loss: 0.2971
Epoch 12/50
12/12 [==============================] - 0s 10

...

131 - ConstantNaive with avg smape 4.85: 
Model Number: 132 of 153 with model MultivariateRegression for Validation 3


[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-2)]: Done 200 out of 200 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished


132 - MultivariateRegression with avg smape 5.78: 
Model Number: 133 of 153 with model MultivariateRegression for Validation 3
Epoch 1/50
43/43 [==============================] - 5s 6ms/step - loss: 100.0057
Epoch 2/50
43/43 [==============================] - 0s 5ms/step - loss: 95.7816
Epoch 3/50
43/43 [==============================] - 0s 7ms/step - loss: 71.2772
Epoch 4/50
43/43 [==============================] - 0s 7ms/step - loss: 55.2881
Epoch 5/50
43/43 [==============================] - 0s 6ms/step - loss: 40.3635
Epoch 6/50
43/43 [==============================] - 0s 6ms/step - loss: 25.4297
Epoch 7/50
43/43 [==============================] - 0s 7ms/step - loss: 33.6663
Epoch 8/50
43/43 [==============================] - 0s 7ms/step - loss: 26.0808
Epoch 9/50
43/43 [==============================] - 0s 6ms/step - loss: 24.6826
Epoch 10/50
43/43 [==============================] - 0s 6ms/step - loss: 29.9767
Epoch 11/50
43/43 [==============================] - 0s 6ms/step - loss

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=7.92053e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


145 - AverageValueNaive with avg smape 8.28: 
Model Number: 146 of 153 with model AverageValueNaive for Validation 3
146 - AverageValueNaive with avg smape 3.13: 
Model Number: 147 of 153 with model AverageValueNaive for Validation 3
147 - AverageValueNaive with avg smape 3.37: 
Model Number: 148 of 153 with model AverageValueNaive for Validation 3
148 - AverageValueNaive with avg smape 4.7: 
Model Number: 149 of 153 with model AverageValueNaive for Validation 3
149 - AverageValueNaive with avg smape 4.19: 
Model Number: 150 of 153 with model AverageValueNaive for Validation 3
150 - AverageValueNaive with avg smape 3.34: 
Model Number: 151 of 153 with model AverageValueNaive for Validation 3
151 - AverageValueNaive with avg smape 4.82: 
Model Number: 152 of 153 with model AverageValueNaive for Validation 3
152 - AverageValueNaive with avg smape 3.83: 
Model Number: 153 of 153 with model GLM for Validation 3
153 - GLM with avg smape 22.83: 


In [ ]:
prediction = model.predict() 

/usr/local/lib/python3.7/dist-packages/sklearn/_loss/glm_distribution.py:132: RuntimeWarning: divide by zero encountered in true_divide
  return -2 * (y - y_pred) / self.unit_variance(y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_glm/glm.py:42: RuntimeWarning: invalid value encountered in multiply
  temp = d1 * family.deviance_derivative(y, y_pred, weights)
/usr/local/lib/python3.7/dist-packages/sklearn/_loss/glm_distribution.py:331: RuntimeWarning: divide by zero encountered in power
  + np.power(y_pred, 2 - p) / (2 - p)
/usr/local/lib/python3.7/dist-packages/sklearn/_loss/glm_distribution.py:132: RuntimeWarning: divide by zero encountered in true_divide
  return -2 * (y - y_pred) / self.unit_variance(y_pred)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_glm/glm.py:42: RuntimeWarning: invalid value encountered in multiply
  temp = d1 * family.deviance_derivative(y, y_pred, weights)
/usr/local/lib/python3.7/dist-packages/sklearn/_loss/glm_distribut

Epoch 1/50
12/12 [==============================] - 6s 5ms/step - loss: 0.6578
Epoch 2/50
12/12 [==============================] - 0s 6ms/step - loss: 0.6473
Epoch 3/50
12/12 [==============================] - 0s 5ms/step - loss: 0.6134
Epoch 4/50
12/12 [==============================] - 0s 5ms/step - loss: 0.5193
Epoch 5/50
12/12 [==============================] - 0s 5ms/step - loss: 0.3271
Epoch 6/50
12/12 [==============================] - 0s 6ms/step - loss: 0.2772
Epoch 7/50
12/12 [==============================] - 0s 7ms/step - loss: 0.2487
Epoch 8/50
12/12 [==============================] - 0s 5ms/step - loss: 0.2256
Epoch 9/50
12/12 [==============================] - 0s 6ms/step - loss: 0.2255
Epoch 10/50
12/12 [==============================] - 0s 6ms/step - loss: 0.2184
Epoch 11/50
12/12 [==============================] - 0s 6ms/step - loss: 0.2090
Epoch 12/50
12/12 [==============================] - 0s 6ms/step - loss: 0.2014
Epoch 13/50
12/12 [==============================

In [ ]:
forecast = prediction.forecast  

In [ ]:
forecast

,Close
2022-07-02,2207.459979
2022-07-03,2231.689761
2022-07-04,2140.617400
2022-07-05,2130.596341
2022-07-06,2123.970835
